We need to evaluate features we can try and improve and restore for photos, this includes - 
    - Blurriness
    - Blemishes
    - Colour Faded/Low Saturations, etc

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Sharpening Photos 

In [ ]:
photo = cv2.imread('sample-photos/old_photo_01.jpg')

k = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]],dtype=float)
k = k / k.sum()
sharp = cv2.filter2D(photo, -1, k)

cv2.imshow('Original Image', photo)
cv2.imshow('Sharp Photo', sharp)

key = cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

Increasing brightness if brightness is below a certain threshold

In [ ]:
grey = cv2.cvtColor(sharp, cv2.COLOR_BGR2GRAY)

brightness = np.mean(grey) #mean to get brightness
brightened = sharp

# if image has low brightness
if brightness < 100:
    gamma = 1.5
    brightened = np.power(sharp / 255.0, 1 / gamma)
    brightened = np.uint8(brightened * 255)

# showing all for testing purposes
cv2.imshow('Original Image', photo)
cv2.imshow('Sharp Photo', sharp)
cv2.imshow('Brightened Image', brightened)
key = cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

MASKING AND INPAINT 

In [ ]:
def show_image(title, img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 4))
    plt.title(title)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

In [ ]:
#function to check if the image has scratches or damages that need inpainting
def needs_masking(photo):
    #convert to greyscale 
    grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY) 
    #create mask of pixels with brightness in between 110 and 160
    mask_mid = cv2.inRange(grey, 110, 160) 

    #calculate if mask contains damage
    damage_ratio = np.count_nonzero(mask_mid) / mask_mid.size
    
    #if more than 1% damage return true 
    return damage_ratio > 0.01   

In [ ]:
#function to generate a mask and repaint those areas 
def masking(photo):
    #convert to greyscale
    grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
    #select mid-intensity pixels where scratches appear
    mask_mid = cv2.inRange(grey, 110, 160) 
    #create a kernel for morphological filtering
    kernel = np.ones((5,5), np.uint8)
    
    #opening to remove small isolated noise 
    mask_mid = cv2.morphologyEx(mask_mid, cv2.MORPH_OPEN, kernel)
    #closing to fill tiny hiles detected 
    mask_mid = cv2.morphologyEx(mask_mid, cv2.MORPH_CLOSE, kernel)

    #another opening to remove bigger spots(reduce false positives)
    mask_small = cv2.morphologyEx(mask_mid, cv2.MORPH_OPEN, np.ones((9,9), np.uint8))
    #conevet into binary mask
    _, damage_mask = cv2.threshold(mask_small, 127, 255, cv2.THRESH_BINARY)

    #use inpaint to fill in the damaged areas 
    restored = cv2.inpaint(photo, damage_mask, 5, cv2.INPAINT_TELEA)

    return damage_mask, restored
show_image("Original Photo", photo)
show_image("inpaint", masking(photo)[1])


gaussian

In [ ]:
#function to check if the images are noisy enough to need gaussian blur
def needs_gaussian(photo, threshold=300): 
    #convert to greyscale
    grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY) 
    #apply laplacian  to highlight edges and noise 
    lap = cv2.Laplacian(grey, cv2.CV_64F)
    #calcuate variance of the laplacian  
    noise_level = lap.var() 
    
    #if noise is abouve threshold return true 
    return noise_level > threshold

In [ ]:
#function to apply the gaussian blur
def gaussian(photo, ksize=3, sigma=0.5): 
    return cv2.GaussianBlur(photo, (ksize, ksize), sigma) 

show_image("Original Photo", photo) 
show_image("Gaussian Blurred", gaussian(photo))

In [ ]:
# libraries imported

import cv2
import numpy as np
import matplotlib.pyplot as plt

# functions for each processing step
def sharpen_image(photo):
    """Apply sharpening kernel."""
    k = np.array([[0, -1, 0],
                  [-1, 5, -1],
                  [0, -1, 0]], dtype=float)
    k = k / k.sum()
    sharp = cv2.filter2D(photo, -1, k)
    return sharp


def adjust_brightness(photo, threshold=100, gamma=1.5):
    gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)

    if brightness < threshold:
        corrected = np.power(photo / 255.0, 1 / gamma)
        corrected = np.uint8(corrected * 255)
        return corrected
    
    return photo

def colour_correction(photo):
    photo_ycrcb = cv2.cvtColor(photo, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(photo_ycrcb)
    
    cr_corrected = cr - (np.mean(cr) - 128)
    cb_corrected = cb - (np.mean(cb) - 128)

    cr_corrected = np.clip(cr_corrected, 0, 255).astype(np.uint8)
    cb_corrected = np.clip(cb_corrected, 0, 255).astype(np.uint8)

    y_eq = cv2.equalizeHist(y)
    
    img_ycrcb_corrected = cv2.merge((y_eq, cr_corrected, cb_corrected))
    final_img = cv2.cvtColor(img_ycrcb_corrected, cv2.COLOR_YCrCb2BGR)

    return final_img

def masking(photo):
    grey = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

    mask_mid = cv2.inRange(grey, 110, 160) 
    
    kernel = np.ones((5,5), np.uint8)
    mask_mid = cv2.morphologyEx(mask_mid, cv2.MORPH_OPEN, kernel)
    mask_mid = cv2.morphologyEx(mask_mid, cv2.MORPH_CLOSE, kernel)

    mask_small = cv2.morphologyEx(mask_mid, cv2.MORPH_OPEN, np.ones((9,9), np.uint8))
    _, damage_mask = cv2.threshold(mask_small, 127, 255, cv2.THRESH_BINARY)

    restored = cv2.inpaint(photo, damage_mask, 5, cv2.INPAINT_TELEA)
    
    return damage_mask, restored

def colour_correction(photo):
    photo_ycrcb = cv2.cvtColor(photo, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(photo_ycrcb)
    
    cr_corrected = cr - (np.mean(cr) - 128)
    cb_corrected = cb - (np.mean(cb) - 128)

    cr_corrected = np.clip(cr_corrected, 0, 255).astype(np.uint8)
    cb_corrected = np.clip(cb_corrected, 0, 255).astype(np.uint8)

    y_eq = cv2.equalizeHist(y)
    
    img_ycrcb_corrected = cv2.merge((y_eq, cr_corrected, cb_corrected))
    final_img = cv2.cvtColor(img_ycrcb_corrected, cv2.COLOR_YCrCb2BGR)

    return final_img
    

def show_results(original, sharp, brightened, damage_mask, restored):
    """This function displays five images in one figure using matplotlib."""
    plt.figure(figsize=(14, 8))

    plt.subplot(1, 5, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 5, 2)
    plt.imshow(cv2.cvtColor(sharp, cv2.COLOR_BGR2RGB))
    plt.title("Sharpened")
    plt.axis('off')

    plt.subplot(1, 5, 3)
    plt.imshow(cv2.cvtColor(brightened, cv2.COLOR_BGR2RGB))
    plt.title("Brightness Adjusted")
    plt.axis('off')

    plt.subplot(1, 5, 4)
    plt.imshow(damage_mask, cmap='gray')
    plt.title("Damage Mask")
    plt.axis('off')

    plt.subplot(1, 5, 5)
    plt.imshow(cv2.cvtColor(restored, cv2.COLOR_BGR2RGB))
    plt.title("Restored")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    
def show_image(title, img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 4))
    plt.title(title)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()
    
def iria(photo):
    output = photo.copy()

    #check for noise
    if needs_gaussian(output):
        output = gaussian(output)

    #check for damage
    #should apply sharpening after inpainting
    if needs_masking(output):
        mask, output = masking(output)

    return output
result = iria(photo)
show_image("Original Photo", photo)
show_image("Final Result ", result)


# main processing pipeline
photo = cv2.imread('sample-photos/old_photo_01.jpg')

sharp = sharpen_image(photo)
brightened = adjust_brightness(sharp)
damage_mask, restored = masking(photo)

show_results(photo, sharp, brightened, damage_mask, restored)